In [13]:
import time
import string
import random
from collections import defaultdict as dd

In [17]:
class Hangman(object):
    def __init__(self):
        self.full_dictionary_location = "data/dictionary.txt"
        self.full_dictionary = self.build_dictionary(self.full_dictionary_location)

        # initialize a list of english letters, and their final probabilties to 0
        self.alphabets = list(string.ascii_lowercase)
        self.probability = [0] * 26

        # builds n_grams[] list, where n_gram[i] -> i-gram for i in [1,5]
        self.n_grams = self.init_n_grams()

        self.guessed_letters = []
        self.tries = 6

        # list of letters sorted by highest unique counts in the dictionary
        self.frequent_letters = self.get_frequent_letters()

        # weightages assigned to each i-gram in the n-grams
        self.preference = [0, 0.05, 0.15, 0.20, 0.25, 0.35]

    def get_frequent_letters(self):
        # returns a list of alphabets sorted by highest unique counts in the dictionary

        # frequency(x) -> no. of words in which the letter x occurs atleast once
        frequency = dd(int)
        for word in self.full_dictionary:
            for letter in set(word):
                frequency[letter] += 1

        letters = self.alphabets.copy()
        letters.sort(key = lambda x: -frequency[x])
        # sorting letters by highest frequency

        return letters

    def init_n_grams(self):
        # builds self.n_grams
        # one-, two- and three- grams have an additional parameter - length of the word
        # This is done to ensure accurate results when guessing the letters in a short word

        # one-gram - self[1][length of word][letter_1]
        one_gram = dd(lambda: dd(int))

        # two-gram - self[2][length of word][letter_1][letter_2]
        two_gram = dd(lambda: dd(lambda: dd(int)))

        # three-gram - self[3][length of word][letter_1][letter_2][letter_3]
        three_gram = dd(lambda:dd(lambda: dd(lambda: dd(int))))

        # four-gram - self[4][letter_1][letter_2][letter_3][letter_4]
        four_gram = dd(lambda:dd(lambda: dd(lambda: dd(int))))

        # five-gram - self[4][letter_1][letter_2][letter_3][letter_4][letter_5]
        five_gram = dd(lambda: dd(lambda:dd(lambda: dd(lambda: dd(int)))))

        self.n_grams = [0, one_gram, two_gram, three_gram, four_gram, five_gram]

        for word in self.full_dictionary:
            l = len(word)

            # processing 1-grams in word
            for letter in set(word):
                self.n_grams[1][l][letter] += 1

            # processing 2-grams in word
            for i in range(l-1):
                self.n_grams[2][l][word[i]][word[i+1]] += 1

            # processing 3-grams in word
            for i in range(l-2):
                self.n_grams[3][l][word[i]][word[i+1]][word[i+2]] += 1

            # processing 4-grams in word
            for i in range(l-3):
                self.n_grams[4][word[i]][word[i+1]][word[i+2]][word[i+3]] += 1

            # processing 5-grams in word
            for i in range(l-4):
                self.n_grams[5][word[i]][word[i+1]][word[i+2]][word[i+3]][word[i+4]] += 1

        return self.n_grams

    def fetch_value(self, ngram, l, text, letter):
        # fetches the n-gram count for the required word replacing '.' by the given letter

        # replace the unknown letter in the n-gram by the current 'letter'
        text = text.replace('.',letter)

        # return the corresponding n-grams counts of this string
        if ngram == 5:
            return self.n_grams[5][text[0]][text[1]][text[2]][text[3]][text[4]]
        elif ngram == 4:
            return self.n_grams[4][text[0]][text[1]][text[2]][text[3]]
        elif ngram == 3:
            return self.n_grams[3][l][text[0]][text[1]][text[2]]
        elif ngram == 2:
            return self.n_grams[2][l][text[0]][text[1]]
        return self.n_grams[1][l][text[0]]

    def process_n_grams(self, word):
        # processes n-grams from 5 to 1 and builds the self.probability list for max. probable letter

        l = len(word)
        self.probability = [0] * 26

        for n in range(5,0,-1):
            # current frequency & probability of occurence of letter
            letter_count = [0] * 26
            cur_prob = [0] * 26
            
            for i in range(l-n+1):
                # the current n-gram
                text = word[i:i+n]

                # should have one unknown letter
                if text.count('.') != 1:
                    continue

                for j in range(26):
                    # for every alphabet not guessed,
                    # substitute it in the blank position in text and fetch its n-gram count

                    if self.alphabets[j] in self.guessed_letters:
                        # we already guessed this letter
                        continue

                    current_value = self.fetch_value(n,l,text,self.alphabets[j])
                    # current_value -> n_gram count placing j^th alphabet in the blank
                    if current_value > 0:
                        letter_count[j] += current_value

            # divide every letter_count by total count to get the probabilities
            total_count = max(sum(letter_count), 1)
            cur_prob = [letter_count[i]/total_count for i in range(26)]

            # update self.probability by taking into account the weightage of this n-gram
            for i in range(26):
                self.probability[i] += cur_prob[i] * self.preference[n]

        # after processing all n-grams, update self.probability
        total_sum = max(sum(self.probability), 1.0)
        self.probability = [x/total_sum for x in self.probability]

        # retrieve the maximum probability value for any letter that is a potential guess
        max_probability = 0
        for i in range(26):
            if self.alphabets[i] in self.guessed_letters:
                # already guessed this letter
                continue

            if self.probability[i] > max_probability:
                max_probability = self.probability[i]

        # find the letter having the obtained max. probability value
        guess_letter = ''
        for i in range(26):
            if self.alphabets[i] in self.guessed_letters:
                continue

            if self.probability[i] == max_probability:
                current_letter = self.alphabets[i]
                if guess_letter == '':
                    guess_letter = current_letter
                else:
                    # multiple letters having the same max frequency
                    # give preference to the letter that occurs more frequently across all words in the dictionary
                    if self.frequent_letters.index(current_letter) < self.frequent_letters.index(guess_letter):
                        guess_letter = current_letter

        # if guess_letter is still not found, return the most frequently occuring non-guessed letter
        if guess_letter == '':
            for i in range(26):
                if self.frequent_letters[i] not in self.guessed_letters:
                    guess_letter = self.frequent_letters[i]
                    break

        return guess_letter


    def guess(self, word):
        # clean the word to strip off spaces and replace "_" with "."
        word = word[::2].replace('_','.')

        # process every n-gram from 5 -> 1 and return the most probable letter
        guess_letter = self.process_n_grams(word)

        return guess_letter

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary


    def startgame(self,given_word):
        self.guessed_letters = []
        self.probability = [0] * 26
        self.tries = 6

        # convert given word to blanks separated by space
        word = "_"
        word_length = len(given_word)
        for i in range(word_length-1):
            word += " _"

        while self.tries > 0:
            guess_letter = self.guess(word)
            self.guessed_letters.append(guess_letter)
            # append the current guess_letter to guessed_letters

            # check if the letter guessed is present in the given word
            hasLetter = False
            for x in given_word:
                if x == guess_letter:
                    hasLetter = True

            
            if hasLetter == False:
                # the guessed letter is not in word, reduce remaining tries by 1
                self.tries -= 1

            else:
                # the guessed letter is in the word
                # convert the blank occurences of the word to this letter in its positions

                word_now = ""
                for i in range(len(word)):
                    if word[i]==' ' or word[i] != '_':
                        word_now += word[i]
                    else:
                        if given_word[i//2] == guess_letter:
                            word_now += guess_letter
                        else:
                            word_now += word[i]
                word = word_now

            if hasLetter == True:
                print("\nCORRECT GUESS: ",guess_letter)
            else:
                print("\nWRONG GUESS: ",guess_letter)
            print("Tries remaining: {0}.".format(self.tries))
            print("Current word: {0}".format(word))
            print("Letters Guessed: {}".format(self.guessed_letters))

            # check if the game is over (guessed all letters of the word)
            gameOver = True
            for i in range(len(word)):
                if word[i] == '_':
                    gameOver = False

            # break the loop if no more letters to guess
            if gameOver:
                break

        print("\nFinal Word Guess: {}\n".format(word))

        # Check if all letters of the word have been guessed
        success = True
        for i in range(len(word)):
            if word[i] == '_':
                success = False
        
        if success:
            return True
        else:
            return False

In [18]:
h = Hangman()

In [19]:
# Test on a random word

dict_size = len(h.full_dictionary)
word = h.full_dictionary[random.randrange(0,dict_size-1,1)]

print("Given Word: ",word)
if h.startgame(word) == True:
    print("SUCCESS")
else:
    print("FAIL")

Given Word:  cassettes

CORRECT GUESS:  e
Tries remaining: 6.
Current word: _ _ _ _ e _ _ e _
Letters Guessed: ['e']

WRONG GUESS:  r
Tries remaining: 5.
Current word: _ _ _ _ e _ _ e _
Letters Guessed: ['e', 'r']

CORRECT GUESS:  s
Tries remaining: 5.
Current word: _ _ s s e _ _ e s
Letters Guessed: ['e', 'r', 's']

CORRECT GUESS:  a
Tries remaining: 5.
Current word: _ a s s e _ _ e s
Letters Guessed: ['e', 'r', 's', 'a']

WRONG GUESS:  l
Tries remaining: 4.
Current word: _ a s s e _ _ e s
Letters Guessed: ['e', 'r', 's', 'a', 'l']

WRONG GUESS:  d
Tries remaining: 3.
Current word: _ a s s e _ _ e s
Letters Guessed: ['e', 'r', 's', 'a', 'l', 'd']

WRONG GUESS:  n
Tries remaining: 2.
Current word: _ a s s e _ _ e s
Letters Guessed: ['e', 'r', 's', 'a', 'l', 'd', 'n']

WRONG GUESS:  m
Tries remaining: 1.
Current word: _ a s s e _ _ e s
Letters Guessed: ['e', 'r', 's', 'a', 'l', 'd', 'n', 'm']

WRONG GUESS:  p
Tries remaining: 0.
Current word: _ a s s e _ _ e s
Letters Guessed: ['e', 'r'